In [ ]:
!pip install qdrant_client
!pip install datasets
!pip install tdqm
!pip install pymilvus
!pip install faiss
!pip install accelerate
!pip install transformers
!pip install -U bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft
!pip install flash_attn
!pip install langchain
!pip install langchain_community
!pip install pymorphy2

In [ ]:
from tqdm import tqdm
from qdrant_client import QdrantClient
from qdrant_client.http import models
from datasets import load_dataset
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from statistics import mean

In [ ]:
qclient = QdrantClient(":memory:")
qclient.create_collection(
    collection_name='rus_xquadqa',
    vectors_config = models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE
    ),
)

True

In [ ]:
dataset = load_dataset('bearberry/rus_xquadqa', split='train') # question, metadata

dataset

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name = 'intfloat/multilingual-e5-large',
    # берем эмбаддинги из модели e5, так как в ней оптимально
    # разбиваются русские слова для хранения в векторной бд
    model_kwargs={"device":"cuda"}, # cpu?
    )

In [ ]:
def vectorize_batch(sentences):
    return embeddings.embed_documents(sentences)
def vectorize_query(query):
    return embeddings.embed_query(query)

In [ ]:
contexts = dataset["context"]
cur_id = 0
for i in tqdm(range(len(contexts))):
    chunks_check = [chunk["chunk"] for chunk in contexts[i]]
    chunks_batch = []
    for chunk in chunks_check:
        # отсекаем несодержательные чанки, длиной менее 20 символов
        # т.к. они засоряют базу
        if len(chunk) < 20:
            continue
        # Далее выполняем проверку на нахождение в базе. Не уверен в
        # оптимальности этого, но пока нет времени разобраться как улучшить
        # и оптимизировать
        points = qclient.query_points(
            collection_name='rus_xquadqa',
            query=vectorize_query(chunk),
            limit=1,
            with_vectors=False,
            with_payload=True
        ).points
        if points:
            chunk_found = points[0].payload["chunk"]
            if chunk != chunk_found:
                chunks_batch.append(chunk)
        else:
            chunks_batch.append(chunk)
    # Добавляем в базу, если батч не пустой
    if chunks_batch:
        batch_len = len(chunks_batch)
        vectors_batch = np.array(vectorize_batch(chunks_batch))
        ids = [j for j in range(cur_id,cur_id + batch_len)]
        cur_id += batch_len
        qclient.upsert(
            collection_name='rus_xquadqa',
            points=models.Batch(
                payloads=[{
                    "chunk":chunks_batch[j]
                } for j in range(batch_len)],
                vectors=[v.tolist() for v in vectors_batch],
                ids=ids,
            )
    )

In [ ]:
# model_name = 'openchat/openchat-3.5-1210'
# or
model_name = 'RefalMachine/RuadaptQwen2.5-7B-Instruct-1M'

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation='sdpa', device_map='auto')
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
pipeline_kwargs={"max_new_tokens": 128, "do_sample": True, "temperature": 0.2}
pipe = pipeline("text-generation",model=model, tokenizer=tokenizer, **pipeline_kwargs)
hugging_face_pipeline = HuggingFacePipeline(pipeline=pipe)

In [ ]:
def relevant_info_qdrant(query,qclient):
    relevant_points = qclient.query_points(
        collection_name='rus_xquadqa',
        query=vectorize_query(query),
        limit=5,
        with_vectors=False,
        with_payload=True
    ).points
    relevant_results = []
    for point in relevant_points:
        relevant_results.append(point.payload["chunk"])
    return relevant_results

In [ ]:
# relevant_docs
# relevant_points.[0].payload["context"][1]["chunk"]

In [ ]:
def apply_chat(messages):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt
chain = apply_chat | hugging_face_pipeline


In [ ]:
# query = "Как изменилось население Варшавы в 19 веке"
# results_text = '\n\n'.join(relevant_info_qdrant(query,qclient))
# query = dataset["question"][i]
# results_text = '\n\n'.join(relevant_info_qdrant(query,qclient))
# prompt = f"Привет, как дела?"
# chat = [
#     {"role": "user", "content": prompt},
# ]

In [ ]:
# res= chain.invoke(chat) # res: GPT4 Correct User, GPT4 Correct Assistant for openchat, assistant\n for ruadaptqwen
# res


NameError: name 'chat' is not defined

In [ ]:
# query = "защита Пэнтерс Пробоул?"
# # Это проверка базы данных
# # query = dataset["question"][7]
# print(query)
# relevant_points = qclient.query_points(
#         collection_name='rus_xquadqa',
#         query=vectorize_query(query),
#         limit=5,
#         with_vectors=False,
#         with_payload=True
#     ).points
# relevant_points

In [ ]:
#rus-x
# working with dataset["question"][:10]

answers = []

for i in tqdm(range(10)):
    query = dataset["question"][i]
    results_text = '\n\n'.join(relevant_info_qdrant(query,qclient))
    prompt = f"Дай только ответ на вопрос, без пояснений. \n\nРелевантная информация по вопросу: {results_text}\n\nВопрос: {query}"
    chat = [
        {"role": "user", "content": prompt},
    ]
    res= chain.invoke(chat) # res: GPT4 Correct User, GPT4 Correct Assistant
    # For opehchat use below
    # answers.append(res[res.rfind("GPT4 Correct Assistant: ") + len("GPT4 Correct Assistant: "):])
    # For Qwen use below
    answers.append(res[res.rfind("assistant\n") + len("assistant\n"):])

for i in range(10):
    print(f"""{answers[i]}, correct_ans: {dataset["answers"][i]}""")

In [ ]:
def get_acc_metrics(test,ans,question,):
    stats = []
    for i in range(len(test)):
        state = 0
        for a in ans[i]:
            if test[i].find(a) != -1:
                state = 1
        stats.append(state)
        # print(question[i])
        print(f"""{question[i]} llm_ans: {answers[i]}, correct_ans: {dataset["answers"][i]}, state: {state}""")
    return mean(stats)

In [ ]:
print("Accuracy: ",get_acc_metrics(answers,dataset["answers"],dataset["question"]))

openchat

Сколько очков уступила защита Пэнтерс?, llm_ans: 308 очков, correct_ans: ['308'], state: 1

Сколько мешков за карьеру было у Джареда Аллена?, llm_ans: 136, correct_ans: ['136'], state: 1

Сколько блокировок записал на свой счет Люк Кикли?, llm_ans: 118, correct_ans: ['118'], state: 1

Сколько мячей перехватил Джош Норман?, llm_ans: 4, correct_ans: ['4', 'четыре'], state: 1

Кто больше записал на свой счет мешков в команде в этом сезоне?, llm_ans: Кейван Шорт, correct_ans: ['Кейван Шорт'], state: 1

Сколько перехватов приписывают защите Пэнтерс в 2015 году?, llm_ans: 24, correct_ans: ['24'], state: 1

Кто был лидером Пэнтерс по мешкам?, llm_ans: Кейван Шорт, correct_ans: ['Кейван Шорт'], state: 1

Сколько игроков защиты Пэнтерс было выбрано для Пробоула?, llm_ans: Два, correct_ans: ['4', 'четыре'], state: 0

Сколько вынужденных потерь мяча имел Томас Дэвис?, llm_ans: 2, correct_ans: ['4', 'четыре'], state: 0

У какого игрока было больше всего перехватов в течение сезона?, llm_ans: Джош Норман, correct_ans: ['Курт Колеман'], state: 0

Accuracy: 0.7

Qwen-3B

Сколько очков уступила защита Пэнтерс? llm_ans: Защита Пэнтерс уступила 308 очков., correct_ans: ['308'], state: 1

Сколько мешков за карьеру было у Джареда Аллена? llm_ans: Джаред Аллен пять раз становился участником Пробоула и за карьеру набрал 136 мешков., correct_ans: ['136'], state: 1

Сколько блокировок записал на свой счет Люк Кикли? llm_ans: Люк Кикли лидировал в команде по блокировкам и записал 118 блокировок., correct_ans: ['118'], state: 1

Сколько мячей перехватил Джош Норман? llm_ans: Шесть., correct_ans: ['4', 'четыре'], state: 0

Кто больше записал на свой счет мешков в команде в этом сезоне? llm_ans: Кейван Шорт лидирует в команде по количеству мешков., correct_ans: ['Кейван Шорт'], state: 1

Сколько перехватов приписывают защите Пэнтерс в 2015 году? llm_ans: Защита Пэнтерс приписывает 24 перехвата., correct_ans: ['24'], state: 1

Кто был лидером Пэнтерс по мешкам? llm_ans: Томас Дэвис был лидером Пэнтерс по мешкам., correct_ans: ['Кейван Шорт'], state: 0

Сколько игроков защиты Пэнтерс было выбрано для Пробоула? llm_ans: 2, correct_ans: ['4', 'четыре'], state: 0

Сколько вынужденных потерь мяча имел Томас Дэвис? llm_ans: Томас Дэвис имел 4 вынужденных потери мяча., correct_ans: ['4', 'четыре'], state: 1

У какого игрока было больше всего перехватов в течение сезона? llm_ans: Курт Колемана, correct_ans: ['Курт Колеман'], state: 1

Accuracy: 0.7

RuadaptQwen-7B-Instruct

Сколько очков уступила защита Пэнтерс? llm_ans: 308 очков., correct_ans: ['308'], state: 1

Сколько мешков за карьеру было у Джареда Аллена? llm_ans: 136 мешков., correct_ans: ['136'], state: 1

Сколько блокировок записал на свой счет Люк Кикли? llm_ans: 118, correct_ans: ['118'], state: 1

Сколько мячей перехватил Джош Норман? llm_ans: 6 (четыре перехвата, включая два тачдауна), correct_ans: ['4', 'четыре'], state: 1

Кто больше записал на свой счет мешков в команде в этом сезоне? llm_ans: Марио Эдисон - 61⁄2 мешка., correct_ans: ['Кейван Шорт'], state: 0

Сколько перехватов приписывают защите Пэнтерс в 2015 году? llm_ans: 24 перехватов., correct_ans: ['24'], state: 1

Кто был лидером Пэнтерс по мешкам? llm_ans: Кейван Шорт - лидировал в команде с 11 мешками., correct_ans: ['Кейван Шорт'], state: 1

Сколько игроков защиты Пэнтерс было выбрано для Пробоула? llm_ans: 5, correct_ans: ['4', 'четыре'], state: 0

Сколько вынужденных потерь мяча имел Томас Дэвис? llm_ans: 4, correct_ans: ['4', 'четыре'], state: 1

У какого игрока было больше всего перехватов в течение сезона? llm_ans: Джош Норман - 88 перехватов., correct_ans: ['Курт Колеман'], state: 0

Accuracy:  0.7